In [29]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import make_classification
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation

def generate_input(df, window_radius = 3):
    _data = []
    for _, item in df.iterrows():
        seq = item.sequence
        length = len(seq)
        
        seq = ("_" * window_radius) + seq + ("_" * window_radius) #add spacer
        for resn in range(length):
            _in = list(seq[resn:resn+window_radius*2+1])
            _data.append(_in)
    return _data

def generate_label(df):
    label = []
    for _, item in df.iterrows():
        ss = item.label
        for resn, _label in enumerate(ss):
            label.append(int(_label))
    return np.array(label)


if __name__ == "__main__":
    

    ###### 1. data preparation ######
    
    # read csv files
    train_val_df = pd.read_csv('train.csv')
    test_df      = pd.read_csv('test.csv') 

    # split into train dataset and validation dataset (not train-test splitting)
    train_df, val_df = train_test_split(train_val_df, random_state=0)

    # extract subsequence
    window_radius = 3
    train_data_ = generate_input(train_df, window_radius)
    val_data_   = generate_input(val_df, window_radius)
    test_data_  = generate_input(test_df, window_radius) 
    
    # encode an amino acids sequence into a numerical vector
    # MUST use the same transformer for all data without refit 
    transformer = OneHotEncoder().fit(train_data_)
    train_data  = transformer.transform(train_data_)
    val_data    = transformer.transform(val_data_)
    test_data   = transformer.transform(test_data_)

    # extract label information
    # Note: NO LABEL INFORMATION for test dataset
    train_label = generate_label(train_df)
    val_label   = generate_label(val_df)
    # test_label = None


    # rename for interpretability
    X_train, Y_train = train_data, train_label
    X_val,   Y_val   = val_data,   val_label
    X_test           = test_data

  

In [31]:
    model = Sequential()
    model.add(Dense(200, input_dim = 153, activation = 'relu')) 
    model.add(Dense(300, activation = 'relu'))
    model.add(Dense(200, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    
    model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
    
    history = model.fit(X_train, Y_train, epochs=150, batch_size=2000, validation_data=(X_val, Y_val))  


Epoch 1/150
559/559 [==============================] - 13s 20ms/step - loss: 0.4695 - accuracy: 0.7888 - val_loss: 0.4330 - val_accuracy: 0.8056
Epoch 2/150
559/559 [==============================] - 13s 19ms/step - loss: 0.4274 - accuracy: 0.8078 - val_loss: 0.4324 - val_accuracy: 0.8064
Epoch 3/150
559/559 [==============================] - 12s 19ms/step - loss: 0.4235 - accuracy: 0.8106 - val_loss: 0.4302 - val_accuracy: 0.8063
Epoch 4/150
559/559 [==============================] - 12s 19ms/step - loss: 0.4201 - accuracy: 0.8117 - val_loss: 0.4302 - val_accuracy: 0.8069
Epoch 5/150
559/559 [==============================] - 13s 19ms/step - loss: 0.4174 - accuracy: 0.8131 - val_loss: 0.4305 - val_accuracy: 0.8065
Epoch 6/150
559/559 [==============================] - 13s 20ms/step - loss: 0.4139 - accuracy: 0.8154 - val_loss: 0.4328 - val_accuracy: 0.8057
Epoch 7/150
559/559 [==============================] - 13s 20ms/step - loss: 0.4113 - accuracy: 0.8160 - val_loss: 0.4336 - val_ac

559/559 [==============================] - 12s 19ms/step - loss: 0.2477 - accuracy: 0.8972 - val_loss: 0.7528 - val_accuracy: 0.7563
Epoch 114/150
559/559 [==============================] - 12s 19ms/step - loss: 0.2473 - accuracy: 0.8970 - val_loss: 0.7605 - val_accuracy: 0.7595
Epoch 115/150
559/559 [==============================] - 12s 19ms/step - loss: 0.2463 - accuracy: 0.8980 - val_loss: 0.7616 - val_accuracy: 0.7532
Epoch 116/150
559/559 [==============================] - 12s 19ms/step - loss: 0.2462 - accuracy: 0.8979 - val_loss: 0.7604 - val_accuracy: 0.7519
Epoch 117/150
559/559 [==============================] - 12s 19ms/step - loss: 0.2462 - accuracy: 0.8979 - val_loss: 0.7653 - val_accuracy: 0.7548
Epoch 118/150
559/559 [==============================] - 12s 19ms/step - loss: 0.2445 - accuracy: 0.8986 - val_loss: 0.7650 - val_accuracy: 0.7582
Epoch 119/150
559/559 [==============================] - 12s 19ms/step - loss: 0.2445 - accuracy: 0.8986 - val_loss: 0.7679 - val_ac

In [52]:
    predicted = model.predict(X_test)
    predicted = np.array(predicted).flatten()
    print(predicted)

[2.7121190e-15 3.0844718e-02 5.3759813e-03 ... 3.5771638e-02 1.0001516e-01
 6.1597434e-15]


In [53]:
    sequence_id_list    = []
    residue_number_list = []
    for _, item in test_df.iterrows():
        sequence_id = item.sequence_id
        sequence    = item.sequence
        for i, aa in enumerate(sequence):
            sequence_id_list.append(sequence_id)
            residue_number_list.append(i+1) #0-origin to 1-origin

    predicted_df = pd.DataFrame.from_dict({
        "sequence_id": sequence_id_list,
        "residue_number": residue_number_list,
        "predicted_value": predicted,
        })
    predicted_df.to_csv('output_5_layer_nn.csv', index=None)